In [183]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import time

def is_element_visible(element):
    return element.is_displayed() and element.is_enabled()

def main():
    options = Options()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    url = "https://steamdb.info/sales/"
    driver.get(url)

    time.sleep(5)

    try:
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".app .b")))
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#DataTables_Table_0 > tbody > tr")))
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".app .dt-type-numeric")))

    except Exception as e:
        print(f"Erro ao localizar elementos: {e}")
        driver.quit()
        return

    try:
        games = driver.find_elements(By.CSS_SELECTOR,  ".app .b")
        prices = driver.find_elements(By.CSS_SELECTOR, "#DataTables_Table_0 > tbody > tr > td:nth-child(5)")
        discounts = driver.find_elements(By.CSS_SELECTOR, "#DataTables_Table_0 > tbody > tr > td:nth-child(4)")

        print(f"Elementos encontrados: Jogos: {len(games)}, Preços: {len(prices)}, Descontos: {len(discounts)}")
    except Exception as e:
        print(f"Erro ao extrair dados: {e}")
        driver.quit()
        return

    game_names = []
    game_prices = []
    discount_percentages = []
    
    for game, price, discount in zip(games, prices, discounts):
        game_names.append(game.text)
        
        price_text = price.text.strip().replace("R", "").replace("$", "").replace(" ", "").replace(",", ".").replace("%", "")
        try:
            game_prices.append(float(price_text))
        except ValueError as e:
            print(f"Erro ao converter o preço: {price_text}. Erro: {e}")
            game_prices.append(None)

        discount_text = discount.text.strip().replace("R", "").replace("$", "").replace(" ", "").replace(",", ".").replace("%", "")
        discount_percentages.append(discount_text)

    driver.quit()

    print(f"Número de jogos encontrados: {len(game_names)}")
    print(f"Número de preços encontrados: {len(game_prices)}")
    print("Nomes dos Jogos:", game_names)
    print("Preços dos Jogos:", game_prices)
    print("Descontos:", discount_percentages)

    data = {
        "game_name": game_names,
        "price": game_prices,
        "discount": discount_percentages
    }

    df = pd.DataFrame(data)

    key_path = "C:/jsonchave/beanalytic-441123-c371efee4e63.json"
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    dataset_id = "beanalytic-441123.steam_data"
    table_id = f"{dataset_id}.steam_sales"

    if not df.empty:
        job_config = bigquery.LoadJobConfig(
            schema=[
                bigquery.SchemaField("game_name", "STRING"),
                bigquery.SchemaField("price", "float"),
                bigquery.SchemaField("discount", "STRING"),
            ],
            write_disposition="WRITE_TRUNCATE",
        )

        try:
            job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
            job.result()
            print("Dados carregados no BigQuery com sucesso!")
        except Exception as e:
            print(f"Ocorreu um erro ao carregar os dados: {e}")
    else:
        print("O DataFrame está vazio. Verifique se os seletores estão corretos e se os dados estão sendo extraídos corretamente.")
